**Tools for NLP**

Regex and tokenizer

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [2]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [3]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [4]:
sen_df = spark.createDataFrame([
    (0, 'Hi I heard about Spark'),
    (1, 'I wish java could use case classes'),
    (2, 'Logistic,regression,models,are,neat')
], ['id', 'sentence'])

In [5]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [6]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [8]:
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words',
                                 pattern='\\W')

In [9]:
count_tokens = udf(lambda words:len(words), IntegerType())

In [10]:
tokenized = tokenizer.transform(sen_df)

In [12]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [13]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [14]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



Stopwords remover

In [15]:
from pyspark.ml.feature import StopWordsRemover

In [16]:
sentenceDataFrame = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green','horse']),
    (1, ['Mary', 'had', 'a', 'little', 'lamb'])
], ['id', 'tokens'])

In [17]:
sentenceDataFrame.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [18]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [19]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



n-gram

In [20]:
from pyspark.ml.feature import NGram

In [21]:
wordDataFrame = spark.createDataFrame([
    (0, ['Hi', 'I', 'heard', 'about', 'Spark']),
    (1, ['I', 'wish', 'Java', 'could', 'use', 'case', 'classes']),
    (2, ['Logistic', 'regression', 'models', 'are', 'neat'])
], ['id', 'words'])

In [22]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')

In [24]:
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



TF-IDF

In [25]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [26]:
sentenceData = spark.createDataFrame([
    (0.0, 'Hi I heard about Spark'),
    (0.0, 'I wish Java could use case classes'),
    (1.0, 'Logistic regression models are neat')
], ['label', 'sentence'])

In [27]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [28]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [29]:
words_data = tokenizer.transform(sentenceData)

In [30]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish Java could...|[i, wish, java, c...|
|  1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [31]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [32]:
featurized_data = hashing_tf.transform(words_data)

In [33]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [34]:
idf_model = idf.fit(featurized_data)

In [35]:
rescaled_data = idf_model.transform(featurized_data)

In [37]:
rescaled_data.select('label', 'features').show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262144,[18700,19...|
|  0.0|(262144,[19036,20...|
|  1.0|(262144,[46243,58...|
+-----+--------------------+



Count vectorizer

In [38]:
from pyspark.ml.feature import CountVectorizer

In [39]:
df = spark.createDataFrame([
    (0, 'a b c'.split(' ')),
    (1, 'a b b c a'.split(' '))
], ['id', 'words'])

In [40]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [41]:
cv = CountVectorizer(inputCol='words', outputCol='features',
                     vocabSize=3, minDF=2.0)

In [43]:
model = cv.fit(df)

In [44]:
result = model.transform(df)

In [46]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+



**Real dataset**

In [47]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [48]:
data = spark.read.csv('./datasets/SMSSpamCollection', inferSchema=True, sep='\t')

In [49]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [50]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [51]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [52]:
from pyspark.sql.functions import length

In [53]:
data = data.withColumn('length', length(data['text']))

In [54]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [55]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [56]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizerModel, IDF, StringIndexer

In [57]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')

ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [58]:
from pyspark.ml.feature import VectorAssembler

In [59]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [64]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline

In [63]:
nb = NaiveBayes()

In [65]:
pipeline = Pipeline(stages=[ham_spam_to_numeric,
                            tokenizer,
                            stop_remove,
                            count_vec,
                            idf,
                            clean_up])

In [66]:
cleaner = pipeline.fit(data)

In [67]:
clean_data = cleaner.transform(data)

In [68]:
clean_data = clean_data.select('label', 'features')

In [69]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [70]:
train, test = clean_data.randomSplit([0.7, 0.3])
train.count(), test.count()

(3877, 1697)

In [71]:
spam_detector = nb.fit(train)

In [72]:
test_results = spam_detector.transform(test)

In [73]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-792.74527907568...|[1.0,3.6708810588...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1065.2728516387...|[1.0,3.6125494907...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-569.55509925376...|[0.99999999999997...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-537.76170093368...|[1.0,8.6450086643...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1364.9529530666...|[1.0,1.3261477098...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-689.62896551995...|[1.0,5.3837785193...|       0.0|
|  0.0|(13424,[0,1,17,19...|[-822.41090644872...|[1.0,1.9890935592...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-862.20915514414...|[1.0,3.4711078466...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-970.08067493190...|[1.0,8.8886464229...|       0.0|
|  0.0|(13424,[0

In [74]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [75]:
acc_eval = MulticlassClassificationEvaluator()

In [76]:
acc = acc_eval.evaluate(test_results)

In [77]:
print('ACC of NB model')
print(acc)

ACC of NB model
0.9172222758380549
